## Import

In [1]:
import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.examples.tutorials.mnist.input_data import read_data_sets

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
print(tf.__version__)

1.4.1


In [3]:
mnist = read_data_sets('data', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


## Define Graph and Training

In [4]:
def weight_variable(shape, name):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name)


def bias_variable(shape, name):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name)

In [14]:
graph = tf.Graph()

with graph.as_default():
    x = tf.placeholder(tf.float32, [None, 784], name="x")
    y = tf.placeholder(tf.float32, [None, 10], name="y")

    with tf.name_scope("Layer1"):
        W_fc1 = weight_variable([784, 128], name='W_fc1')
        b_fc1 = bias_variable([128], name='b_fc1')
        a_fc1 = tf.add(tf.matmul(x, W_fc1), b_fc1, name="zscore")
        h_fc1 = tf.nn.relu(a_fc1)
    with tf.name_scope("Layer2"):
        W_fc2 = weight_variable([128, 64], name='W_fc2')
        b_fc2 = bias_variable([64], name='b_fc2')
        a_fc2 = tf.add(tf.matmul(h_fc1, W_fc2), b_fc2, name="zscore")
        h_fc2 = tf.nn.relu(a_fc2)
    with tf.name_scope("OuputLayer"):
        W_fc3 = weight_variable([64, 10], name='W_fc3')
        b_fc3 = bias_variable([10], name='b_fc3')
        logits = tf.add(tf.matmul(h_fc2, W_fc3), b_fc3, name="logits")
    y_pred = tf.argmax(logits, 1, name='y_pred')
        
    with tf.name_scope("Loss"):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, 
                                                                logits=logits)
        loss = tf.reduce_mean(cross_entropy, name="cross_entropy_loss")
    
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss, name="train_step")
    
    with tf.name_scope("Prediction"): 
        correct_prediction = tf.equal(y_pred, 
                                      tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

In [15]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()

    for i in range(1, 20001):
        images, labels = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: images, y: labels})
        if i % 1000 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: images, y: labels})
            print('step %d, training accuracy %g' % (i, train_accuracy))
    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))
    chkp_path = saver.save(sess, "./my-model/model.ckpt")

step 1000, training accuracy 0.9
step 2000, training accuracy 0.96
step 3000, training accuracy 0.94
step 4000, training accuracy 0.98
step 5000, training accuracy 0.94
step 6000, training accuracy 0.92
step 7000, training accuracy 0.96
step 8000, training accuracy 0.94
step 9000, training accuracy 0.94
step 10000, training accuracy 0.94
step 11000, training accuracy 0.98
step 12000, training accuracy 0.94
step 13000, training accuracy 0.98
step 14000, training accuracy 0.92
step 15000, training accuracy 0.96
step 16000, training accuracy 0.96
step 17000, training accuracy 1
step 18000, training accuracy 0.98
step 19000, training accuracy 1
step 20000, training accuracy 1
test accuracy 0.9726


In [16]:
chkp_path

'./my-model/model.ckpt'

## Remove Training Nodes and Freeze Graph

In [21]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, chkp_path)
    out_nodes = [y_pred.op.name]
    sub_graph_def = graph_util.remove_training_nodes(sess.graph_def)
    sub_graph_def = graph_util.convert_variables_to_constants(sess, sub_graph_def, out_nodes)

INFO:tensorflow:Restoring parameters from ./my-model/model.ckpt
INFO:tensorflow:Froze 6 variables.
Converted 6 variables to const ops.


In [22]:
# save freezed graph
graph_path = tf.train.write_graph(sub_graph_def, 
                                  "./my-model", "simple_mnist.pb", 
                                  as_text=False)
print('written graph to: %s' % graph_path)

written graph to: ./my-model/simple_mnist.pb


## Quantization

In [23]:
# with TransformGraph
from tensorflow.tools.graph_transforms import TransformGraph

In [26]:
quant_graph_def = TransformGraph(sub_graph_def, [], out_nodes, 
                                 ["quantize_weights", "quantize_nodes", "sort_by_execution_order"])

In [28]:
with tf.gfile.FastGFile('my-model/quant_mnist.pb', 'wb') as fid:
    fid.write(quant_graph_def.SerializeToString())